In [1]:
# All imports
from pathlib import Path
import warnings
from typing import Any, Tuple, Union
import cv2
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt

In [4]:
BASE: Path = Path("../bleaching_data_RS")
IMG_DIR: Path = BASE / "images"
MASKS_BLEACHED_DIR: Path = BASE / "masks_bleached"
MASKS_UNBLEACHED_DIR: Path = BASE / "masks_non_bleached"


def find_mask_triple(img_path: Path) -> Union[Tuple[Path, Any, Any], Tuple[Path, None, None]]:
    """
    Group the original and mask image files based on their stem.
    The stem is the original file name without the file extension. 
    As this is the same for all 3 images, it will select them all. 

    Args:
        img_path (Path): file path of an image in the "images" folder

    Returns:
        tuple[Path, Any, Any] | tuple[Path, None, None]: tuple of file paths for all equal images (original + 2 masks)
    """
    image_stem = img_path.stem 

    bleached_path = MASKS_BLEACHED_DIR / f"{image_stem}_bleached.png"
    unbleached_path = MASKS_UNBLEACHED_DIR / f"{image_stem}_non_bleached.png"
    if bleached_path.exists() and unbleached_path.exists(): # Check that all 3 versions of the image are there, and group them
        return img_path, bleached_path, unbleached_path
    return img_path, None, None

# Build the dataset
triple: list[Any] = []
for img_path in IMG_DIR.glob("*.jpg"):
    img_path, bleached_path, non_bleached_path = find_mask_triple(img_path)
    if bleached_path is None or non_bleached_path is None:  # Warning if there is a missing mask
        print(f"[WARN] Missing masks for {img_path.name}")
        continue
    triple.append((img_path, bleached_path, non_bleached_path))

print(f"Found {len(triple)} image/mask triples.")

Found 658 image/mask triples.


In [ ]:
def srgb_to_linear(color_channel_array: npt.NDArray[np.uint8]) -> npt.NDArray[np.float32]:
    """
    To invert the nonlinear scale, a specific formula needs to be applied based on the value of the pixel. 
    If the value is lower or equal to 0.04045, then the value should be divided by 12.92.
    If the value is higher than 0.04045, then v = ((v+0.055) / 1.055) ^ 2.4

    Args:
        color_channel_array (npt.NDArray[np.uint8]): 2D array of a color channel

    Returns:
        npt.NDArray[np.float32]: 2D array of the linearized color channel.
    """
    x = color_channel_array.astype(np.float32) / 255.0  # First convert the pixel values to [0,1] scale
    return np.where(x <= 0.04045, x / 12.92, ((x + 0.055) / 1.055) ** 2.4).astype(np.float32)

def robust_unit(x: np.ndarray, mask: np.ndarray | None) -> np.ndarray:
    # Ensure float32
    x = x.astype(np.float32, copy=False)
    H, W = x.shape[:2]

    if mask is None:
        mask = np.ones((H, W), dtype=bool)
    else:
        mask = mask.astype(bool, copy=False)
        if mask.shape[:2] != (H, W):
            mask = np.ones((H, W), dtype=bool)  # last-resort fallback

    vals = x[mask]
    vals = vals[np.isfinite(vals)]

    if vals.size < 4:
        base = x[np.isfinite(x)]
        if base.size < 4:
            return np.zeros_like(x, dtype=np.float32)
        p1, p99 = np.percentile(base, [1, 99])
    else:
        p1, p99 = np.percentile(vals, [1, 99])

    # Cast percentiles to float32 to prevent upcasting x to float64
    p1  = np.float32(p1)
    p99 = np.float32(p99)
    denom = np.float32(max(p99 - p1, 1e-6))

    # In-place normalize and clip to avoid extra big allocations
    y = x.copy()                   # one working buffer
    y -= p1                        # in-place
    y /= denom                     # in-place
    np.clip(y, 0.0, 1.0, out=y)    # in-place
    return y

def create_all_scores(bgr_img: npt.NDArray[np.uint8], mask_matrix: npt.NDArray[np.bool_]) -> Tuple[Any, Any, Any, Any, Any, Any]:
    """
    Converts an image and its mask into all the visual traits we want to investigate. 

    Args:
        bgr_img (npt.NDArray[np.uint8]): 3D matrix of (HxWxc) with c being the color channel.
        mask_matrix (npt.NDArray[np.uint8]): 2D matrix of (HxW).

    Returns:
        Tuple[Any, Any, Any, Any, Any, Any]: A tuple with bgr_GW, LAB_img, raw_red_score, albedo_score, luminance_score, saturation_score.
    """
    # Convert input image to GWWB
    b, g, r = cv2.split(bgr_img.astype(np.float32))
    mean_b, mean_g, mean_r = b.mean(), g.mean(), r.mean()
    mean_value = (mean_b + mean_g + mean_r) / 3
    b *= mean_value / (mean_b + 1e-6)
    g *= mean_value / (mean_g + 1e-6)
    r *= mean_value / (mean_r + 1e-6)
    bgr_GW = cv2.merge([b, g, r])
    bgr_GW = np.clip(bgr_GW, 0, 255).astype(np.float32)

    # Convert sRGB values to a perceptually linear scale, and compute raw-red and albedo scores
    bgr_linear = srgb_to_linear(bgr_GW)
    sum_linear = bgr_linear.sum(axis=-1)
    raw_red_score = (bgr_linear[..., 2] / (sum_linear + 1e-6)) 
    albedo_score = (sum_linear / 3.0).astype(np.float32)

    # Convert the GWWB images to CIELAB, and equalize the luminance using the CLAHE method
    LAB_img = cv2.cvtColor(bgr_GW.astype(np.uint8), cv2.COLOR_BGR2LAB) # This function requires the pixel values to be integers.
    luminance, a_channel, b_channel = cv2.split(LAB_img)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    L_equalized = clahe.apply(luminance)

    # Create luminance and saturation scores, and normalize them to percentiles
    luminance_score = L_equalized.astype(np.float32) / 255
    a_channel_normalized = a_channel.astype(np.float32) - 128
    b_channel_normalized = b_channel.astype(np.float32) - 128
    saturation_score = np.sqrt(a_channel_normalized**2 + b_channel_normalized**2)
    luminance_score = robust_unit(luminance_score, mask_matrix)
    saturation_score = robust_unit(saturation_score, mask_matrix)


    return bgr_GW, LAB_img, raw_red_score, albedo_score, luminance_score, saturation_score

In [84]:
def create_ECDFs(path_triples: list[tuple[Path, Path, Path]])-> dict[str, npt.NDArray[np.float32]]: 
    """
    For each image in the given dataset, the whiteness score is calculated for the coral pixels.
    Outliers (= values outside winsorization range) are clipped. 
    Then a histogram of the whiteness values is created. 
    Input setting "weighting" decides if all pixels have equal weight, or if all images do.
    Then an ECDF is created from all histograms.

    Args:
        path_triples (list[tuple[Path, Path, Path]]): Paths to the images.

    Raises:
        ValueError: Raise error if empty score array is found.

    Returns:
        dict[str, npt.NDArray[np.float32]]: Dictionary with bin edges and cdfs of each score.
    """
    # Iterate over all images in the given dataset
    for img_path, mb_path, mn_path in path_triples:
        img_bgr: npt.NDArray[np.uint8] = cv2.imread(str(img_path), cv2.IMREAD_COLOR)            
        img_HEIGHT, img_WIDTH = img_bgr.shape[:2]           

        # Load masks and create coral mask.
        mask_bleached = load_mask(mb_path, (img_HEIGHT, img_WIDTH))
        mask_non_bleached = load_mask(mn_path, (img_HEIGHT, img_WIDTH))
        all_coral_mask: npt.NDArray[np.bool_] = (mask_bleached | mask_non_bleached) if (mask_bleached is not None and mask_non_bleached is not None) else None          
        if all_coral_mask is None or not all_coral_mask.any():
            continue
        
        # Calculate all scores
        _, _, raw_red_score, albedo_score, luminance_score, saturation_score = create_all_scores(img_bgr, all_coral_mask)


        def get_ecdf(score: npt.NDArray[np.float32]) -> Tuple[npt.NDArray[np.float32], npt.NDArray[np.float32]]:
            # Initialize variables
            nr_of_bins = 4096
            bin_heights = np.zeros(nr_of_bins, dtype=np.float64)
            bin_edges = None

            # 1D matrix of whiteness scores, excludes 0 values of non coral pixels
            score_flattened = score[all_coral_mask]  # coral-only 1D
            if score_flattened.size == 0:
                raise ValueError(f"Empty score array detected.")

            # Values outside the winsor range get clipped to the nearst number in the range (to prevfent extreme points from dominating the histogram)
            lo, hi = np.percentile(score_flattened, (0.05, 0.95))
            score_flattened = np.clip(score_flattened, lo, hi)

            # bin_edges stores the bin edges as an 1D array. 
            count, bin_edges = np.histogram(score_flattened, bins=nr_of_bins, range=(0.0, 1.0))

            # Each image has the same influence on the ECDF. Big images will have the same influence as small images.
            sum = count.sum()
            if sum > 0:
                bin_heights += count / sum

            # Sum all histograms, and normalize to keep the whiteness scores in [0,1]
            cdf = np.cumsum(bin_heights) / bin_heights.sum()
            return (bin_edges, cdf)
    
    # Create separate ECDF for each score
    raw_red_bin_edges, raw_red_cdf = get_ecdf(raw_red_score)
    print("raw red done")
    albedo_bin_edges, albedo_cdf = get_ecdf(albedo_score)
    print("albedo done")
    luminance_bin_edges, luminance_cdf = get_ecdf(luminance_score)
    print("luminance done")
    saturation_bin_edges, saturation_cdf = get_ecdf(saturation_score)

    # Save cdf and input values as a dictionary
    return {"raw_red_bin_edges": raw_red_bin_edges, "raw_red_cdf": raw_red_cdf, 
            "albedo_bin_edges": albedo_bin_edges, "albedo_cdf": albedo_cdf, 
            "luminance_bin_edges": luminance_bin_edges, "luminance_cdf": luminance_cdf, 
            "saturation_bin_edges": saturation_bin_edges, "saturation_cdf": saturation_cdf}

In [111]:
test_set = triple[:50]